# Interest Rate Futures Pricing and DV01
This notebook calculates a model price and DV01 for a Treasury bond futures contract. It does so using QuantLib and publicly availalble data. The example focuses on a futures contract on the 30-year bond but the data sources and mechanics would work similarly for others.

This example utilizes the QuantLib Python Cookbook written by Goutham Balaraman and Luigi Ballabio. This is a very useful book and I recommend it.

Treasury rate data is obtained through Quandl. This data is Constant Maturity Treasury data instead of actual Treasury Bills/Notes/Bonds because I haven't found that available freely.

The list of deliverable Treasuries, as part of the futures contract, are obtained from QuickStrick via the CME website.

# Libraries

In [1]:
import quandl
from QuantLib import *
import math
import pandas as pd
import numpy as np
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common import keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from collections import OrderedDict
import os.path
import textwrap
import getpass

import warnings

# Market Data
## Constant Maturity Treasuries

In [2]:
quandl.ApiConfig.api_key = 'hog8enrHbV2oq7yP7cNf'
Tbill4WK = quandl.get('FRED/DTB4WK')
Tbill1MO = quandl.get('FRED/DTB3')
Tbill6MO = quandl.get('FRED/DTB6')
Tbill1YR = quandl.get('FRED/DTB1YR')
Tnote2YR = quandl.get('FRED/DGS2')
Tnote3YR = quandl.get('FRED/DGS3')
Tnote5YR = quandl.get('FRED/DGS5')
Tnote7YR = quandl.get('FRED/DGS7')
Tnote10YR = quandl.get('FRED/DGS10')
Tbond20YR = quandl.get('FRED/DGS20')
Tbond30YR = quandl.get('FRED/DGS30')

In [3]:
treas_dict = {'Tbill4WK': Tbill4WK, 'Tbill1MO': Tbill1MO, 'Tbill6MO': Tbill6MO,
              'Tbill1YR': Tbill1YR, 'Tnote2YR': Tnote2YR, 'Tnote3YR': Tnote3YR,
              'Tnote5YR': Tnote5YR, 'Tnote7YR': Tnote7YR, 'Tnote10YR': Tnote10YR,
              'Tbond20YR': Tbond20YR, 'Tbond30YR': Tbond30YR}
periods_dict = {'Tbill4WK': Period(4, Weeks), 'Tbill1MO': Period(1, Months), 'Tbill6MO': Period(6, Months),
                'Tbill1YR': Period(1, Years), 'Tnote2YR': Period(2, Years), 'Tnote3YR': Period(3, Years),
                'Tnote5YR': Period(5, Years), 'Tnote7YR':Period(7, Years), 'Tnote10YR': Period(10, Years),
                'Tbond20YR': Period(20, Years),'Tbond30YR': Period(30, Years)}

In [4]:
max_treas = min([treas_dict[treas].index[-1] for treas in list(treas_dict.keys())])
treasuries = pd.concat(treas_dict.values(), axis=1).dropna()
treasuries.columns = list(treas_dict.keys())
treas_curr = treasuries.iloc[treasuries.index==max_treas]
treas_curr

,Tbill4WK,Tbill1MO,Tbill6MO,Tbill1YR,Tnote2YR,Tnote3YR,Tnote5YR,Tnote7YR,Tnote10YR,Tbond20YR,Tbond30YR
Date,,,,,,,,,,,
2017-07-27,0.99,1.09,1.11,1.19,1.36,1.52,1.84,2.12,2.32,2.68,2.93


# Futures Price
The code below is used to log-in and scrape the QuickStrike tool provided by CME's website. It extracts the current price of the futures contract on the 30 Year Treasury.

In [5]:
def get_treasury_futures():
    warnings.filterwarnings('ignore')
    browser = webdriver.Chrome('C:/local/chrome/chromedriver.exe')
    browser.get('https://cmegroup.quikstrike.net/Account/Login.aspx?ReturnUrl=/User/QuikStrikeView.aspx')

    # Browser options
    login = browser.find_element_by_name('ctl00$MainContent$ucLoginControl$LoginUser$UserName')
    login.send_keys('jnewkirk')
    psswd = browser.find_element_by_name('ctl00$MainContent$ucLoginControl$LoginUser$Password')
    psswd.send_keys('l7kme8BFFTnM')
    submit = browser.find_element_by_name('ctl00$MainContent$ucLoginControl$LoginUser$LoginButton')
    submit.send_keys(keys.Keys.RETURN)
    browser.find_element_by_name('chkAccept').click()
    browser.find_element_by_name('btnContinue').click()
    browser.get(
        "https://cmegroup.quikstrike.net/User/QuikStrikeView.aspx?sc=1&viewitemid=TreasuryDeliveryBasket&mode=&pid=2")
    html = browser.page_source
    soup = BeautifulSoup(html, 'lxml')

    name = []
    symbol = []
    price = []
    ctd_coupon = []
    ctd_maturity_date = []
    ctd_issue_date = []
    ctd_forward_yield = []
    ctd_DV01 = []
    otr_coupon = []
    otr_maturity_date = []
    otr_issue_date = []
    otr_forward_yield = []
    otr_DV01 = []

    for tr in soup.find_all('tr')[2:]:
        tds = tr.find_all('td')
        name.append((tds[0].text).strip())
        symbol.append((tds[1].text).strip())
        price.append(float((tds[2].text)))
        ctd_coupon.append((tds[3].text).strip())
        ctd_maturity_date.append(datetime.strptime(((tds[4].text).strip()), "%m/%d/%Y"))
        ctd_issue_date.append(datetime.strptime(((tds[5].text).strip()), "%m/%d/%Y"))
        ctd_forward_yield.append(float((tds[6].text)[:-1]))
        ctd_DV01.append(float((tds[7].text)[1:]))
        otr_coupon.append((tds[8].text).strip())
        otr_maturity_date.append(datetime.strptime(((tds[9].text).strip()), "%m/%d/%Y"))
        otr_issue_date.append(datetime.strptime(((tds[10].text).strip()), "%m/%d/%Y"))
        otr_forward_yield.append(float((tds[11].text)[:-1]))
        otr_DV01.append(float((tds[12].text)[1:]))

    browser.close()

    columns = OrderedDict()
    columns = {'instrument_name': name, 'symbol': symbol, 'price': price, 'ctd_coupon': ctd_coupon,
               'ctd_maturity_date': ctd_maturity_date, 'ctd_issue_date': ctd_issue_date,
               'ctd_forward_yield': ctd_forward_yield, 'ctd_DV01': ctd_DV01,
               'otr_coupon': otr_coupon, 'otr_maturity_date': otr_maturity_date, 'otr_issue_date': otr_issue_date,
               'otr_forward_yield': otr_forward_yield, 'otr_DV01': otr_DV01}
    df = pd.DataFrame(columns, columns=columns.keys())
    warnings.filterwarnings('always')
    return df

treasury_futures = get_treasury_futures()
treasury_futures_summary = treasury_futures.loc[:,['instrument_name', 'symbol', 'price']]
treasury_futures_summary
futures_price = treasury_futures_summary[treasury_futures_summary['instrument_name']=='30 Yr']
futures_price = np.asscalar(futures_price['price'].values)
futures_price

153.09

# Futures Deliverables
The code below is used to log-in and scrape the QuickStrike tool provided by CME's website. It is hard-coded to only pull the deliverables for the futures contract on the 30-year bond.


In [6]:
def get_treasury_futures_deliverables():
    warnings.filterwarnings('ignore')
    browser = webdriver.Chrome('C:/local/chrome/chromedriver.exe')
    browser.get('https://cmegroup.quikstrike.net/Account/Login.aspx?ReturnUrl=/User/QuikStrikeView.aspx')
    browser.implicitly_wait(3)

    # Browser options
    login = browser.find_element_by_name('ctl00$MainContent$ucLoginControl$LoginUser$UserName')
    login.send_keys('jnewkirk')
    psswd = browser.find_element_by_name('ctl00$MainContent$ucLoginControl$LoginUser$Password')
    psswd.send_keys('l7kme8BFFTnM')
    submit = browser.find_element_by_name('ctl00$MainContent$ucLoginControl$LoginUser$LoginButton')
    submit.send_keys(keys.Keys.RETURN)
    browser.find_element_by_name('chkAccept').click()
    browser.find_element_by_name('btnContinue').click()
    browser.get(
        "https://cmegroup.quikstrike.net/User/QuikStrikeView.aspx?sc=1&viewitemid=TreasuryDeliveryBasket&mode=&pid=2")

    element = browser.find_element_by_id("ctl00_MainContent_ucViewControl_TreasuryDeliveryBasket_lvTabs_ctrl1_lbFuture")
    element.click()
    tableID = 'ctl00_MainContent_ucViewControl_TreasuryDeliveryBasket_ucBasketUSU7_ucDeliverables_lvDeliverables_tblDeliverables'
    WebDriverWait(browser, 5).until(
        expected_conditions.presence_of_element_located(
            (By.ID, tableID )))
    html = browser.page_source
    soup = BeautifulSoup(html, 'lxml')
    coupon = []
    maturity_date= []
    mid_price = []
    CUSIP = []
    issue_date = []
    table = soup.find(lambda tag: tag.name == 'table' and tag.has_key('id') and tag['id'] == tableID)
    for tr in table.find_all('tr')[2:]:
        tds = tr.find_all('td')
        coupon.append(float(tds[0].text))
        maturity_date.append(datetime.strptime((tds[1].text.strip()), "%m/%d/%Y"))
        mid_price.append(float(tds[2].text))
        CUSIP.append(tds[3].text.strip()[:-1])
        issue_date.append(datetime.strptime((tds[4].text.strip()), "%m/%d/%Y"))
    browser.close()

    columns = OrderedDict()
    columns = {'coupon': coupon, 'maturity': maturity_date, 'price': mid_price,
               'CUSIP': CUSIP,'issue': issue_date
                }
    df = pd.DataFrame(columns, columns=columns.keys())
    warnings.filterwarnings('always')
    return df

deliverables = get_treasury_futures_deliverables()
deliverables

,coupon,maturity,price,CUSIP,issue
0,4.500,2036-02-15,128.4766,912810FT0,2006-02-15
1,5.000,2037-05-15,136.5313,912810PU6,2007-08-15
2,4.750,2037-02-15,132.5938,912810PT9,2007-02-15
3,4.500,2038-05-15,128.7266,912810PX0,2008-08-15
4,4.375,2038-02-15,126.6719,912810PW2,2008-02-15
5,4.500,2039-08-15,128.8047,912810QC5,2009-08-17
6,4.625,2040-02-15,131.0625,912810QE1,2010-02-16
7,4.250,2039-05-15,124.5625,912810QB7,2009-05-15
8,4.375,2039-11-15,126.7188,912810QD3,2009-11-16
9,4.750,2041-02-15,133.7734,912810QN1,2011-02-15


In [7]:
basket_maturity_dates = [Date(dt.day, dt.month, dt.year) for dt in deliverables['maturity']]
basket_issue_dates = [Date(dt.day, dt.month, dt.year) for dt in deliverables['issue']]
basket = list(zip(deliverables['coupon'], basket_issue_dates, basket_maturity_dates, deliverables['price']))

# Calculation
## Calibration of Yield Curve

In [8]:
treas_curr_date = pd.Timestamp(treas_curr.index.values[0])
calc_date = Date(treas_curr_date.day, treas_curr_date.month, treas_curr_date.year)
calc_date
Settings.instance().evaluationDate = calc_date
day_count_tbill = Actual360()
day_count = ActualActual()
calendar = UnitedStates()
business_convention = Following
end_of_month = False
settlement_days = 0
face_amount = 100
coupon_frequency = Period(Semiannual)
#coupon_frequency = Period(6, Months)

In [9]:
issue_dates = [calc_date]*len(treas_curr.columns)
maturity_dates = [calendar.advance(calc_date,period) for period in periods_dict.values()]

In [10]:
yields = [np.asscalar(treas_curr.iloc[:,col].values)/100 for col in range(len(treas_curr.columns))]
prices = [100*(1-yields[i]*day_count_tbill.yearFraction(calc_date,maturity_dates[i])) 
          if day_count.yearFraction(calc_date,maturity_dates[i])<=1 else 100 
          for i in range(len(treas_curr.columns))]
print(prices)
coupons = [yields[i] if day_count.yearFraction(calc_date,maturity_dates[i])>1 else 0 
           for i in range(len(treas_curr.columns))]
print(coupons)

[99.923, 99.90311111111112, 99.42649999999999, 98.79347222222222, 100, 100, 100, 100, 100, 100, 100]
[0, 0, 0, 0, 0.013600000000000001, 0.0152, 0.0184, 0.0212, 0.0232, 0.0268, 0.029300000000000003]


In [11]:
bond_helpers = []
for coupon, issue_date, maturity_date, price in zip(coupons, issue_dates, maturity_dates, prices):
    schedule = Schedule(calc_date, maturity_date, coupon_frequency, calendar, 
                        business_convention, business_convention, DateGeneration.Backward, False)
    helper = FixedRateBondHelper(QuoteHandle(SimpleQuote(price)),
        settlement_days,
        face_amount,
        schedule,
        [coupon],
        day_count,
        business_convention
        )
    bond_helpers.append(helper)

In [12]:
yield_curve = PiecewiseCubicZero(calc_date, bond_helpers, day_count)
yield_curve_handle = YieldTermStructureHandle(yield_curve)
discount_factors = [yield_curve.discount(d) for d in maturity_dates]
df_df = (pd.DataFrame.from_dict(dict(zip(maturity_dates, discount_factors)), orient='index'))
df_df.columns = ['Discount Factors']
df_df

,Discount Factors
"August 24th, 2017",0.999230
"August 28th, 2017",0.999031
"January 29th, 2018",0.994265
"July 27th, 2018",0.987935
"July 29th, 2019",0.973159
"July 27th, 2020",0.955472
"July 27th, 2022",0.911931
"July 29th, 2024",0.861081
"July 27th, 2027",0.791043
"July 27th, 2037",0.577300


## Cheapest to Deliver

In [13]:
def create_tsy_security(bond_issue_date,
                        bond_maturity_date,
                        coupon_rate,
                        coupon_frequency=Period(6, Months),
                        day_count=ActualActual(),
                        calendar=UnitedStates()
                        ):
    face_value = 100.
    settlement_days = 0
    schedule = Schedule(bond_issue_date,
        bond_maturity_date,
        coupon_frequency,
        calendar,
        ModifiedFollowing,
        ModifiedFollowing,
        DateGeneration.Forward,
        False)
    security = FixedRateBond(settlement_days,
        face_value,
        schedule,
        [coupon_rate],
        day_count
        )
    return security

In [14]:
print(futures_price)

153.09


In [15]:
securities = []
min_basis = 100; min_basis_index=-1
for i, b in enumerate(basket):
    coupon, issue, maturity, price = b
    s = create_tsy_security(issue, maturity, coupon/100.0)
    bond_engine = DiscountingBondEngine(yield_curve_handle)
    s.setPricingEngine(bond_engine)
    cf = BondFunctions.cleanPrice(s, 0.06, day_count, Compounded, Semiannual, calc_date)/100
    adjusted_futures_price = futures_price *cf
    basis = price - adjusted_futures_price
    if basis< min_basis:
        min_basis = basis
        min_basis_index = i
    securities.append((s, cf))
ctd_info = basket[min_basis_index]
ctd_bond, ctd_cf = securities[min_basis_index]
ctd_price = ctd_info[2]
print(min_basis)
print(ctd_cf)
print(ctd_info[0])
print(ctd_info[1])
print(ctd_info[2])
print(ctd_info[3])


0.885929638543
0.833435693784421
4.5
February 15th, 2006
February 15th, 2036
128.4766


# Futures Price and DV01

In [16]:
delivery_date = Date(15, 9, 2017)
futures = FixedRateBondForward(calc_date, delivery_date, Position.Long, 0.0, settlement_days,
                               day_count, calendar, business_convention, ctd_bond,
                               yield_curve_handle, yield_curve_handle)
ctd_bond_yield = ctd_bond.bondYield(ctd_bond.dayCounter(),Compounded, Semiannual)
y = InterestRate(ctd_bond_yield,ctd_bond.dayCounter(),Compounded, Semiannual)
ctd_bond_duration = BondFunctions.duration(ctd_bond, y)
ctd_bond_dv01 = ctd_bond_duration*ctd_info[3]*.0001
futures_dv01 = ctd_bond_dv01/ctd_cf*1000

In [17]:
model_futures_price = futures.cleanForwardPrice()/ctd_cf
model_futures_price

152.84767338473003

In [18]:
print('DV01 of CTD bond: {:9.6f}'.format(ctd_bond_dv01))
print('DV01 of futures contract: {:9.6f}'.format(futures_dv01))
units = 15000
print('Dollar Rho for 1% move: {:,.0f}'.format(units*futures_dv01*100))

DV01 of CTD bond:  0.168841
DV01 of futures contract: 202.584824
Dollar Rho for 1% move: 303,877,236
